## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2025">The Repo</a>.  

Once you have updated your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://github.com/apps/databricks">Databricks App on Github</a> or by cloning the repo to your laptop and then uploading the final_project directory and its contents to your workspace using file imports.  Your choice.

<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches to triggering your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)
- [In class examples - Spark Structured Streaming Performance](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/2638424645880316?o=1093580174577663)

### Be sure your project runs end to end when *Run all* is executued on this notebook! (7 points)

### This project is worth 25% of your final grade.
- DSCC-202 Students have 55 possible points on this project (see points above and the instructions below)
- DSCC-402 Students have 60 possible points on this project (one extra section to complete)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    # Optimize the tables
    optimize_table(BRONZE_DELTA)
    optimize_table(SILVER_DELTA)
    optimize_table(GOLD_DELTA)
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here (2 points)
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, when, isnan, to_timestamp, explode, udf, regexp_replace, expr, lower, trim
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, DoubleType, IntegerType, FloatType
from delta.tables import DeltaTable
import delta
import mlflow
import mlflow.pyfunc
from mlflow.pyfunc import spark_udf
from transformers import pipeline
import os
import json
import time
import shutil
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import time
spark.conf.set("spark.sql.shuffle.partitions", "12")

## 2.0 Define and execute utility functions (3 points)
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE
files = dbutils.fs.ls(TWEET_SOURCE_PATH)
print(f"Count of files in the input directory: {len(files)}")
display(spark.read.json(files[0].path))

## 3.0 Transform the Raw Data to Bronze Data using a stream  (8 points)
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using delta lake to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defined in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE

raw_data_schema = StructType([StructField("date", StringType(), True), StructField("user", StringType(), True), StructField("text", StringType(), True), StructField("sentiment", StringType(), True), StructField("source_file", StringType(), True), StructField("processing_time", TimestampType(), True)])

bronze_stream = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.partitionColumns", "")
        .option("maxFilesPerTrigger", 10)
        .schema(raw_data_schema)
        .load(TWEET_SOURCE_PATH)
)


(
    bronze_stream.writeStream
        .format("delta")
        .option("checkpointLocation", BRONZE_CHECKPOINT)
        .option("mergeSchema", "true")
        .outputMode("append")
        .trigger(processingTime="30 seconds")
        .queryName("bronze_stream")
        .start(BRONZE_DELTA)
)

## 4.0 Transform the Bronze Data to Silver Data using a stream (5 points)
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE

bronze_stream =(
    spark.readStream
    .format("delta")
    .load(BRONZE_DELTA)
)

silver_stream = (
    bronze_stream
    .withColumn("timestamp", to_timestamp("processing_time"))
    .withColumn("mention", explode(expr("filter(split(text, ' '), x -> x LIKE '@%')")))
    .filter(col("mention").startswith("@"))                   
    .withColumn("cleaned_text", regexp_replace(col("text"), "@\\w+", "")) 
    .select("timestamp", "mention", "cleaned_text", "sentiment")
)

(
    silver_stream.writeStream
    .format("delta")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .outputMode("append")
    .trigger(processingTime="30 seconds")
    .queryName("silver_stream")
    .start(SILVER_DELTA)
)

## 5.0 Transform the Silver Data to Gold Data using a stream (7 points)
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ENTER YOUR CODE HERE

hf_pipeline = pipeline("sentiment-analysis", model=HF_MODEL_NAME)

def predict_sentiment_struct(text):
    result = hf_pipeline([text])[0]
    label = result["label"].upper()
    score = result["score"]
    sentiment_id = 1 if label == "POSITIVE" else 0
    return(label, score, sentiment_id)

sentiment_schema = StructType([
    StructField("predicted_sentiment", StringType())
    ,StructField("predicted_score", DoubleType())
    ,StructField("predicted_sentiment_id", IntegerType())
])

predict_sentiment_udf = udf(predict_sentiment_struct, sentiment_schema)

silver_stream = (
    spark.readStream
    .format("delta")
    .load(SILVER_DELTA)
)

gold_stream = (
    silver_stream
    .withColumn("sentiment_struct", predict_sentiment_udf(col("cleaned_text")))
    .select(
        "timestamp", "mention", "cleaned_text", "sentiment",
        col("sentiment_struct.predicted_sentiment"),
        col("sentiment_struct.predicted_score"),
        col("sentiment_struct.predicted_sentiment_id")
    )
)

(
    gold_stream.writeStream
    .format("delta")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .outputMode("append")
    .trigger(processingTime="30 seconds")
    .queryName("gold_stream")
    .start(GOLD_DELTA)
)

## 6.0 Monitor your Streams (5 points)
- Setup a loop that runs at least every 10 seconds
- Print a timestamp of the monitoring query along with the list of streams, rows processed on each, and the processing time on each
- Run the loop until all of the data is processed (0 rows read on each active stream)
- Plot a line graph that shows the data processed by each stream over time
- Plot a line graph that shows the average processing time on each stream over time

In [0]:
# ENTER CODE HERE

def get_streaming_stats():
    data = []
    start_time = None

    required_queries = {"bronze_stream", "silver_stream", "gold_stream"}

    print("Waiting for all streams to become active + initialized...")

    while True:
        active_streams = {q.name: q for q in spark.streams.active}
        
        # Considering streams that have at least one progress update
        initialized = {
            name for name, q in active_streams.items()
            if name in required_queries and q.recentProgress and len(q.recentProgress) > 0
        }

        print(f"Initialized streams: {initialized}")
        
        if initialized == required_queries:
            print("All required streams are initialized.")
            break

        time.sleep(60)

    for q in spark.streams.active:
        for p in q.recentProgress:
            timestamp = datetime.strptime(p["timestamp"], "%Y-%m-%dT%H:%M:%S.%fZ")

            if start_time is None:
                start_time = timestamp

            elapsed_time = (timestamp - start_time).total_seconds()

            processing_time = p["durationMs"].get("addBatch", None) if "durationMs" in p else None

            data.append({
                "query": q.name,
                "elapsed_time": elapsed_time,
                "input_rows": p.get("numInputRows", 0),
                "processing_time": processing_time
            })

    return pd.DataFrame(data)

# Monitoring Loop
print("Monitoring active streams...")

row_count = 1
i = 0
while row_count != 0:
    time.sleep(10)

    stats = get_streaming_stats()
    if not stats.empty:
        latest_stats = stats.sort_values("elapsed_time").groupby("query").last().reset_index()
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Elapsed: {i*10}s")
        for _, row in latest_stats.iterrows():
            print(f"  {row['query']}: {row['input_rows']} rows, {row['processing_time']} ms processing time")
        row_count = latest_stats["input_rows"].sum()
        i += 1

        if all(latest_stats["input_rows"] == 0):
            print("All streams have finished - exit monitoring.")
            break


df = get_streaming_stats()

# Plotting
if not df.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    # Plot 1: Processing Time
    for q in df["query"].unique():
        subset = df[df["query"] == q]
        ax1.plot(subset["elapsed_time"], subset["processing_time"], marker='o', linestyle='-', label=q)
    ax1.set_ylabel("Processing Time (ms)")
    ax1.set_title("Spark Streaming Processing Time")
    ax1.legend()
    ax1.grid(True)

    # Plot 2: Input Rows
    for q in df["query"].unique():
        subset = df[df["query"] == q]
        ax2.plot(subset["elapsed_time"], subset["input_rows"], marker='o', linestyle='-', label=q)
    ax2.set_xlabel("Elapsed Time (seconds)")
    ax2.set_ylabel("Input Rows")
    ax2.set_title("Spark Streaming Input Rows")
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

## 7.0 Bronze Data Exploratory Data Analysis (5 points)
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE

BRONZE_DF = spark.read.format("delta").load(BRONZE_DELTA)
tweet_count = BRONZE_DF.count()
print(f"Total number of tweets: {tweet_count}")

null_exprs = []
for c in BRONZE_DF.columns:
    dtype = BRONZE_DF.schema[c].dataType
    if isinstance(dtype, (DoubleType, FloatType)):
        null_exprs.append(count(when(col(c).isNull() | isnan(col(c)), c)).alias(c))
    else:
        null_exprs.append(count(when(col(c).isNull(), c)).alias(c))

null_check = BRONZE_DF.select(null_exprs)
print("Null/NaN counts per column:")
null_check.show()

user_counts = BRONZE_DF.groupBy("user").count().orderBy(desc("count"))
user_counts.show()

mention_col = "text"
with_mentions = BRONZE_DF.filter(col(mention_col).contains("@")).count()
without_mentions = BRONZE_DF.filter(~col(mention_col).contains("@")).count()
print(f"Tweets with @mention: {with_mentions}")
print(f"Tweets without @mention: {without_mentions}")

top20_users = user_counts.limit(20).toPandas()

plt.figure(figsize=(12, 6))
bars = plt.bar(top20_users["user"], top20_users["count"], color="skyblue")
plt.title("Top 20 Users by Tweet Count")
plt.ylabel("Tweet Count")
plt.xlabel("User")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8.0 Capture the accuracy metrics from the gold table in MLflow  (4 points)
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the model name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE

GOLD_DF = spark.read.format("delta").load(GOLD_DELTA)

pdf = GOLD_DF.select(
    "sentiment",
    "predicted_sentiment",
    "predicted_sentiment_id",
    "predicted_score"
).toPandas()

y_true = pdf["sentiment"].str.upper()
y_pred = pdf["predicted_sentiment"].str.upper()

precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")

cm = confusion_matrix(y_true, y_pred, labels=["POSITIVE", "NEGATIVE", "NEU"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["POS", "NEG", "NEU"])

fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax)
plt.tight_layout()
conf_matrix_path = "/tmp/confusion_matrix.png"
plt.savefig(conf_matrix_path)
plt.close()

with mlflow.start_run(run_name="gold_metrics_logging"):
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    mlflow.log_artifact(conf_matrix_path)

    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("mlflow_version", mlflow.__version__)

    delta_table = DeltaTable.forPath(spark, GOLD_DELTA)
    delta_version = delta_table.history().select("version").orderBy("version", ascending=False).first()["version"]
    mlflow.log_param("delta_version", delta_version)

## 9.0 Application Data Processing and Visualization (6 points)
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ENTER YOUR CODE HERE

gold_df = spark.read.format("delta").load(GOLD_DELTA)

mention_count = gold_df.filter(col("mention").isNotNull()).count()
print(f"Total number of mentions: {mention_count}")

gold_df = gold_df.withColumn("sentiment", trim(lower(col("sentiment"))))

mention_sentiment_counts = (
    gold_df.filter(col("mention").isNotNull())
    .groupBy("mention")
    .agg(
        count(when(col("sentiment") == "neu", True)).alias("neutral"),
        count(when(col("sentiment") == "positive", True)).alias("positive"),
        count(when(col("sentiment") == "negative", True)).alias("negative")
    )
)

mention_sentiment_counts = mention_sentiment_counts.withColumn(
    "total", col("neutral") + col("positive") + col("negative")
)

mention_sorted = mention_sentiment_counts.orderBy(col("total").desc())
mention_sorted.show()

top20_positive = mention_sorted.orderBy(col("positive").desc()).limit(20).toPandas()
plt.figure(figsize=(12, 6))
plt.bar(top20_positive["mention"], top20_positive["positive"], color='green')
plt.title("Top 20 Mentions with Positive Sentiment")
plt.xlabel("Mention")
plt.ylabel("Positive Tweet Count")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

top20_negative = mention_sorted.orderBy(col("negative").desc()).limit(20).toPandas()
plt.figure(figsize=(12, 6))
plt.bar(top20_negative["mention"], top20_negative["negative"], color='red')
plt.title("Top 20 Mentions with Negative Sentiment")
plt.xlabel("Mention")
plt.ylabel("Negative Tweet Count")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## 10.0 Clean up and completion of your pipeline (3 points)
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook. Note: In the includes there is a variable START_TIME that captures the starting time of the notebook.

In [0]:
# ENTER YOUR CODE HERE

print("Active Streams:")
for q in spark.streams.active:
    print(f" - {q.name}")

for q in spark.streams.active:
    print(f"Stopping stream: {q.name}")
    q.stop()

end_time = time.time() 
elapsed = end_time - START_TIME
print(f"\nTotal Elapsed Time: {elapsed:.2f} seconds")


## 11.0 How Optimized is your Spark Application (Grad Students Only) (5 points)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE